In [1]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

In [2]:
from google.colab import files
uploaded = files.upload()

Saving credit_card_transcations.csv to credit_card_transcations.csv


In [12]:
#create engine to run sql query
engine = create_engine("sqlite:///creditcard.db")


In [13]:
# Load CSV
df_creditcard = pd.read_csv("credit_card_transcations.csv")

In [17]:
# Save DataFrame as a SQL table
df_creditcard.to_sql("credit_card_transcations", engine, index=False, if_exists="replace")

26052

In [18]:
def run_query(query):
    return pd.read_sql(query, engine)

In [19]:
run_query("SELECT * FROM credit_card_transcations limit 5 ;")

,transaction_id,city,transaction_date,card_type,exp_type,gender,amount
0,1,Delhi,29-Oct-14,Gold,Bills,F,82475
1,2,Greater Mumbai,22-Aug-14,Platinum,Bills,F,32555
2,3,Bengaluru,27-Aug-14,Silver,Bills,F,101738
3,4,Greater Mumbai,12-Apr-14,Signature,Bills,F,123424
4,5,Bengaluru,5-May-15,Gold,Bills,F,171574


In [20]:
#cheq row count / basic scanity cheq
run_query("select count(1) as total_transactions from credit_card_transcations;")

,total_transactions
0,26052


In [21]:
# duplicate transaction id if present or not , if no records means no dupliacte records are present
run_query("select transaction_id , count(*) as duplicate_records from credit_card_transcations group by transaction_id having count(*) > 1;")

,transaction_id,duplicate_records


In [32]:
# finding missing or null critical field
query="""select
        sum(case when transaction_id is null then 1 else 0 end ) as missing_txn_id,
        sum(case when amount is null then 1 else 0 end ) as missing_amt,
        sum(case when transaction_date is null then 1 else 0 end) as date_missing
        from credit_card_transcations;"""

run_query(query)

,missing_txn_id,missing_amt,date_missing
0,0,0,0


In [33]:
# finding negative or zero transaction amount
query=""" select *
          from credit_card_transcations
          where amount <= 0;
"""

run_query(query)

,transaction_id,city,transaction_date,card_type,exp_type,gender,amount


In [34]:
# finding data consistency/ future dates or old dates
query=""" select *
          from credit_card_transcations
          where transaction_date > date('now')
          and transaction_date < 2002;
"""

run_query(query)

,transaction_id,city,transaction_date,card_type,exp_type,gender,amount


Business problems 🇰
1- write a query to print top 5 cities with highest spends and their percentage contribution of total credit card spends

2- write a query to print highest spend month and amount spent in that month for each card type

3- write a query to print the transaction details(all columns from the table) for each card type when
it reaches a cumulative of 1000000 total spends(We should have 4 rows in the o/p one for each card type)

4- write a query to find city which had lowest percentage spend for gold card type

5- write a query to print 3 columns:  city, highest_expense_type , lowest_expense_type (example format : Delhi , bills, Fuel)

6- write a query to find percentage contribution of spends by females for each expense type

7- which card and expense type combination saw highest month over month growth in Jan-2014

9- during weekends which city has highest total spend to total no of transcations ratio

10- which city took least number of days to reach its 500th transaction after the first transaction in that city


In [42]:
#  1- write a query to print top 5 cities with highest spends and their percentage contribution of total credit card spends

query= """ with cte1 as (
select city,sum(amount) as total_spend
from credit_card_transcations
group by city)
,total_spent as (select sum(cast(amount as int)) as total_amount from credit_card_transcations)
select cte1.*, round(total_spend*1.0/total_amount * 100,2) as percentage_contribution from
cte1 inner join total_spent on 1=1
order by total_spend desc
limit 5
;

"""

run_query(query)

,city,total_spend,percentage_contribution
0,Greater Mumbai,576751476,14.15
1,Bengaluru,572326739,14.05
2,Ahmedabad,567794310,13.93
3,Delhi,556929212,13.67
4,Kolkata,115466943,2.83


In [48]:
#2- write a query to print the transaction details(all columns from the table) for each card type when
#it reaches a cumulative of  1,000,000 total spends(We should have 4 rows in the o/p one for each card type)

query= """with cte as (
select *,sum(amount) over(partition by card_type order by transaction_date,transaction_id) as total_spend
from credit_card_transcations
--order by card_type,total_spend desc
)
select * from (select *, rank() over(partition by card_type order by total_spend) as rn
from cte where total_spend >= 1000000) a where rn=1"""

run_query(query)

,transaction_id,city,transaction_date,card_type,exp_type,gender,amount,total_spend,rn
0,13432,Delhi,1-Apr-14,Gold,Grocery,M,23697,1000956,1
1,18395,Surandai,1-Apr-14,Platinum,Entertainment,F,290618,1060394,1
2,20660,Jaipur,1-Apr-14,Signature,Bills,F,190198,1073671,1
3,13961,Bengaluru,1-Apr-14,Silver,Entertainment,F,172789,1000757,1


In [50]:
#3- write a query to find city which had lowest percentage spend for gold card type
query="""with cte as (
select city,card_type,sum(amount) as amount
,sum(case when card_type='Gold' then amount end) as gold_amount
from credit_card_transcations
group by city,card_type
limit 1
)
select
city,sum(gold_amount)*1.0/sum(amount) as gold_ratio
from cte
group by city
having count(gold_amount) > 0 and sum(gold_amount)>0
order by gold_ratio;"""

run_query(query)

,city,gold_ratio
0,Achalpur,1.0


In [51]:
# 4 write a query to print 3 columns:  city, highest_expense_type , lowest_expense_type (example format : Delhi , bills, Fuel)

query="""with cte as (
select city,exp_type, sum(amount) as total_amount from credit_card_transcations
group by city,exp_type)
select
city , max(case when rn_asc=1 then exp_type end) as lowest_exp_type
, min(case when rn_desc=1 then exp_type end) as highest_exp_type
from
(select *
,rank() over(partition by city order by total_amount desc) rn_desc
,rank() over(partition by city order by total_amount asc) rn_asc
from cte) A
group by city;"""

run_query(query)

,city,lowest_exp_type,highest_exp_type
0,Achalpur,Entertainment,Grocery
1,Adilabad,Food,Bills
2,Adityapur,Grocery,Food
3,Adoni,Entertainment,Bills
4,Adoor,Bills,Fuel
...,...,...,...
981,Zaidpur,Bills,Food
982,Zamania,Fuel,Entertainment
983,Zira,Grocery,Entertainment
984,Zirakpur,Entertainment,Fuel


In [52]:
#--5- write a query to find percentage contribution of spends by females for each expense type
query="""select exp_type,
sum(case when gender='F' then amount else 0 end)*1.0/sum(amount) as percentage_female_contribution
from credit_card_transcations
group by exp_type
order by percentage_female_contribution desc;"""

run_query(query)

,exp_type,percentage_female_contribution
0,Bills,0.639459
1,Food,0.549053
2,Travel,0.511329
3,Grocery,0.509110
4,Fuel,0.497104
5,Entertainment,0.493729
